 Creates `train.npz` and `test.npz` by making a simle temporal crop from 10am to 4pm across the full dataset.

In [1]:
import xarray as xr
import numpy as np
import pathlib
import datetime
import pandas as pd
import tqdm

In [2]:
SATELLITE_ZARR_PATH = "gs://public-datasets-eumetsat-solar-forecasting/satellite/EUMETSAT/SEVIRI_RSS/v3/eumetsat_seviri_hrv_uk.zarr"

dataset = xr.open_dataset(
    SATELLITE_ZARR_PATH, 
    engine="zarr",
    chunks="auto",  # Load the data as a Dask array
)

print(dataset)


<xarray.Dataset>
Dimensions:  (time: 173624, y: 891, x: 1843)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01T00:05:00 ... 2021-11-07T15:50:00
  * x        (x) float32 2.8e+04 2.7e+04 2.6e+04 ... -1.813e+06 -1.814e+06
    x_osgb   (y, x) float32 dask.array<chunksize=(891, 1843), meta=np.ndarray>
  * y        (y) float32 4.198e+06 4.199e+06 4.2e+06 ... 5.087e+06 5.088e+06
    y_osgb   (y, x) float32 dask.array<chunksize=(891, 1843), meta=np.ndarray>
Data variables:
    data     (time, y, x) int16 dask.array<chunksize=(22, 891, 1843), meta=np.ndarray>


In [3]:
def get_day_slice(date):
    data_slice = dataset.loc[
        {
            # 10am to 4pm
            "time": slice(
                date + datetime.timedelta(hours=10),
                date + datetime.timedelta(hours=16),
            )
        }
    ].isel(
        x=slice(550, 950),
        y=slice(375, 700),
    )
    
    # sometimes there is no data
    if len(data_slice.time) == 0:
        return None
    return data_slice

In [4]:
# it might be worth it to do this in batches of one year... a full download will take at least
# 30 minutes if not hours
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2021, 12, 31)

cur = start_date
days_to_get = []
while cur != end_date + datetime.timedelta(days=1):
    days_to_get.append(cur)
    cur = cur + datetime.timedelta(days=1)

In [5]:
slices = []
for date in tqdm.tqdm(days_to_get):
    slc = get_day_slice(date)
    if slc is None:
        continue
    slices.append(slc)

100%|██████████| 731/731 [00:00<00:00, 817.82it/s]


In [6]:
len(slices)

606

In [ ]:
combined = xr.concat(slices, dim='time')

In [ ]:
# takes a while
times = combined['time'].to_numpy()
x = combined['x'].to_numpy()
x_osgb = combined['x_osgb'].to_numpy()
y = combined['y'].to_numpy()
y_osgb = combined['y_osgb'].to_numpy()
%time data = combined['data'].to_numpy()

In [ ]:
times.shape, data.shape


In [ ]:
# due to some weird things in how we originally made `train.npz` and `test.npz`,
# this will not be equivalent to the datasets we linked in the drive. But they
# should still work.
test_days = 30

np.random.seed(7)
test_dates = np.random.choice(days_to_get, size=test_days, replace=False)

test_indices = []
train_indices = []
for i, t in enumerate(times):
    d = pd.Timestamp(t).date()
    if d in test_dates:
        test_indices.append(i)
    else:
        train_indices.append(i)

In [ ]:
test_data = data[test_indices]
test_times = times[test_indices]

train_data = data[train_indices]
train_times = times[train_indices]

In [ ]:
# save data
p = pathlib.Path(f'train.npz')
if p.exists():
    raise ValueError(f'Path {p} already exists!')

np.savez(
    p,
    times=train_times,
    data=train_data,
)

p = pathlib.Path(f'test.npz')
if p.exists():
    raise ValueError(f'Path {p} already exists!')

np.savez(
    p,
    times=test_times,
    data=test_data,
)

p = pathlib.Path(f'coords.npz')
if p.exists():
    raise ValueError(f'Path {p} already exists!')

np.savez(
    p,
    x=x,
    x_osgb=x_osgb,
    y=y,
    y_osgb=y_osgb,
)